Наброски кода для
0. сведения алломорфов морфем в одно (хотя бы в таблице)
1. подсчёта сочетаний морфем
2. вытаскивания конвербов
3. вытаскивания цепочек частей речи

Все манипуляции можно тестировать на корпусах, которые в почте.

# 0
Действительно надо вытащить всё, положить в словарину и радоваться

Формат:

`корпус{
    название:документ{    # done
        мета:мета,    # done
        текст:[
            предложение{
                слой:[
                    морфемы
                    ],
                перевод:''
            }
        ]
    }
}`

In [1]:
corpora = ['Kamchatka', 'Sebjan']

In [2]:
import os, re, pickle
from pprint import pprint
import numpy as np
import pandas as pd 

In [4]:
def morphs_2_words(line):
    '''берёт на вход массив расчленённой строки, возвращает массив слов'''
    words = []
    word = []
    for morph in line:
        if morph.strip()[0] in '=-':
            word.append(morph)
        else:
            if len(word) > 0:
                words.append(word)
            word = [morph]
    words.append(word)
    return words


def handle_startline(line, res, current_layer):
    line = line.split()
    layer = line[0].strip('\\')
    parted_layers = ['tx', 'mb', 'ge', 'ps']
    if not (len(line) == 1): # чтобы не считать пустые строки
        if layer in parted_layers:
            line_content = morphs_2_words(line[1:]) # делим на слова, состоящие из морфем
        else:
            line_content = [' '.join(line[1:])] # просто целые строки (комментарии и тп)
            current_layer = layer
        if layer in res and res[layer][0] != '':
            res[layer] += line_content
        else:
            res[layer] = line_content
    return res, current_layer


def lines_2_dict(part):
    '''
    i: кусок текста (предложение) в несколько строк, в каждой строке несколько слоёв, и с другими данными предложения
    o: джейсонина вида {'слой': [сл, о, ва], 'слой': содержимое}
    доп. ограничения: длина всех строк-массивов равна
    '''
    res = {}
    lines = [line for line in part.split('\n') if len(line) > 1]
    res['index'] = [lines[0].split('_')[-1]]
    parted_layers = ['tx', 'mb', 'ge', 'ps']
    current_layer = '' # для переносов
    for line in lines[1:]:
        if line.startswith('\\'):
            res, current_layer = handle_startline(line, res, current_layer)
        else:
            if current_layer:
                res[current_layer][0] += ' ' + line
    return res


def check_len(p_sent, fil):
    parted_layers = ['mb', 'ge', 'ps']
    selected_layers = [key for key in p_sent if key in parted_layers and len(p_sent[key])>1]
    # 1. check that the number of words is the same
    lengths = set([len(p_sent[key]) for key in selected_layers])
    if len(lengths) > 1:
        print('Error in {}, here:'.format(fil))
        print(lengths)
        for l in selected_layers:
            print(len(p_sent[l]))
            pprint(p_sent[l])
        return lengths
    # 2. check that morphemes are aligned
    if 'mb' in selected_layers and 'ge' in selected_layers:
        for i in range(len(p_sent['ge'])):
            if len(p_sent['ge'][i]) != len(p_sent['mb'][i]):
                print('што-то слиплось в {}'.format(fil))
                pprint(p_sent)
    return lengths


def make_readable(corp):
    "переводит текст корпусов в удобомашиночитаемую джейсонину"
    folder = 'Corpus_Text_{}_postagged'.format(corp)
    corpus_dict = {}
    for fil in os.listdir(folder):
        if 'pyzhik' in fil:
            continue
        if not fil.endswith('.txt'):
            continue
        with open(os.path.join(folder, fil), 'r') as f:
            text = f.read()
        file_content, text_content = {}, []
        sents = text.split('\id')
        file_content['meta'] = sents[1] # metainfo at the beginning of the file; not parsed
        for sent in sents[2:]:
            sent_content = lines_2_dict(sent)
            check_len(sent_content, fil)
            text_content.append(sent_content)
        file_content['text'] = text_content
        corpus_dict[fil] = file_content
    with open('{}.pickle'.format(corp), 'wb') as f:
        pickle.dump(corpus_dict, f)
    return corpus_dict

In [60]:
for corp in corpora:
    make_readable(corp)

In [3]:
def detect(what, where, replace=False, corpora=corpora, show=True):
    hitss = []
    for corpus in corpora:
        hits = []
        with open('{}.pickle'.format(corpus), 'rb') as f:
            content = pickle.load(f)
        for doc in content:
            for i in range(len(content[doc]['text'])): # looping through sentences
                if not where in content[doc]['text'][i]:
                    continue
                for word in content[doc]['text'][i][where]:
                    if what in word:
                        print('Found in {}, {} at {}'.format(corpus, doc, i))
                        if show:
                            pprint(content[doc]['text'][i])
                        hits.append(doc)
        if len(hits) == 0:
            print('noth found. cool! or not?')
        else:
            inp = input('correct all these ({} hits) & press Enter and I\'ll reload.\n'.format(len(hits)))
            if replace:
                target = replace
                for doc in list(set(hits)):
                    with open(os.path.join('Corpus_Text_{}_postagged'.format(corpus), doc), 'r') as f:
                        text = f.read()
                    a_hits = re.findall('{}\\b'.format(what), text)
                    while len(a_hits) > len(hits):
                        print('what is not accurate. see what\'d be affected')
                        pprint(a_hits[:15])
                        what = input('change the what: ')
                        a_hits = re.findall('{} '.format(what.replace('.', '\.')), text)
                    text = re.sub('{}(\\b)'.format(what.replace('.', '\.')), '{}\\1'.format(target), text)
                    with open(os.path.join('Corpus_Text_{}_postagged'.format(corpus), doc), 'w') as f:
                        f.write(text)
            make_readable(corpus)
            print('done!')
        hitss.append(hits)
    return hitss

In [4]:
detect('-nr', 'ge')

Found in Kamchatka, Solodikov_GA_tabun.txt at 16
{'ELANBegin': ['00:01:46.959'],
 'ELANEnd': ['00:01:53.216'],
 'ELANParticipant': ['GAS'],
 'ev': ['Нян дютки да\uf321ымныди, нян дявышақла эммоттэ.'],
 'ft': ['And having chased them close to the house they come to the place '
        'where one catches reindeer (=djawasak).'],
 'ge': [['and'],
        ['house', '-all'],
        ['bring.closer', '-ant.cvb'],
        ['and'],
        ['catch.hold', '-ep', '-nr', '-loc'],
        ['come', '-gnr', '-nonfut']],
 'gr': ['(15.02.13, questions20, 13:00) dagịmnịdʒị = prigonjaem; ja olenej '
        'prigonjaju = bi ọralbe dagịmnam; ix prignali = ọral dagịmaptịtan; in '
        'the dictionary dagam- = priblizitsja'],
 'index': ['018'],
 'mb': [['ńaːn'],
        ['dʒụː', '-t(E)kI'],
        ['dagịm', '-RIdʒI'],
        ['ńaːn'],
        ['dʒaw', '-E', '-ČEk', '-(dU)LE'],
        ['em', '-WEːČ', '-R(E)']],
 'ps': [['adv'], ['n'], ['v'], ['adv'], ['v'], ['v']],
 'ru': ['И вот до дому пригоняем, и 

KeyboardInterrupt: 

In [43]:
nfs = ['-nonfut3pl', '-nonfut3sg', '-nonfut.3pl', '-nonfut.3sg']
for nf in nfs:
    detect(nf, 'ge', replace='-nonfut', show=False)
print('\n\nregalar nfs done\n')
other_nfs = ['-hab.nonfut3pl', '-hab.nonfut3sg', '-hab.nonfut.3pl', '-hab.nonfut.3sg']
for nf in other_nfs:
    detect(nf, 'ge', replace='-hab.nonfut', show=False)
other_nfs = ['-caus.nonfut3pl', '-caus.nonfut3sg', '-caus.nonfut.3pl', '-caus.nonfut.3sg']
for nf in other_nfs:
    detect(nf, 'ge', replace='-caus.nonfut', show=False)

noth found. cool! or not?
noth found. cool! or not?
noth found. cool! or not?
noth found. cool! or not?
noth found. cool! or not?
noth found. cool! or not?
noth found. cool! or not?
noth found. cool! or not?


regalar nfs done

noth found. cool! or not?
noth found. cool! or not?
noth found. cool! or not?
noth found. cool! or not?
noth found. cool! or not?
Found in Sebjan, SlepcovaNA_her_class.txt at 27
Found in Sebjan, Zaxarova_JP_pear_story.txt at 21
correct all these (2 hits) & press Enter and I'll reload.

done!
noth found. cool! or not?
noth found. cool! or not?
noth found. cool! or not?
noth found. cool! or not?
noth found. cool! or not?
noth found. cool! or not?
noth found. cool! or not?
Found in Sebjan, Krivoshapkina_AE_childhood.txt at 114
Found in Sebjan, Krivoshapkin_DM_Segen.txt at 40
Found in Sebjan, Krivoshapkin_DM_Segen.txt at 40
correct all these (3 hits) & press Enter and I'll reload.

done!
noth found. cool! or not?
noth found. cool! or not?


Здесь повторяю таблицу которая была

In [2]:
import numpy as np
import pandas as pd 

In [159]:
def extract_morphs(corpus):
    with open('{}.pickle'.format(corpus), 'rb') as f:
        content = pickle.load(f)
    morphs = []
    for doc in content:
        for sent in content[doc]['text']:
            if 'mb' in sent and 'ge' in sent and 'ps' in sent:
                for i in range(len(sent['mb'])):
                    morphs.append(tuple(['ROOT', 'ROOT', 'ROOT']))
                    ps = sent['ps'][i][0]
                    for l in range(1, len(sent['mb'][i])):
                        try:
                            morphs.append((sent['mb'][i][l].strip('-='), sent['ge'][i][l].strip('-='), ps))
                        except:
                            print(doc)
                            pprint(sent)
                    morphs.append(tuple(['END', 'END', 'END']))
    with open('{}_morphemes.pickle'.format(corpus), 'wb') as f:
        pickle.dump(morphs, f)
    print(len(morphs))
    pprint(morphs[:15])

In [160]:
for corp in corpora:
    extract_morphs(corp)

105395
[('ROOT', 'ROOT', 'ROOT'),
 ('END', 'END', 'END'),
 ('ROOT', 'ROOT', 'ROOT'),
 ('W', 'acc', 'n'),
 ('END', 'END', 'END'),
 ('ROOT', 'ROOT', 'ROOT'),
 ('END', 'END', 'END'),
 ('ROOT', 'ROOT', 'ROOT'),
 ('RI', 'pst', 'v'),
 ('WUn', 'poss.1pl.ex', 'v'),
 ('END', 'END', 'END'),
 ('ROOT', 'ROOT', 'ROOT'),
 ('t(E)kI', 'all', 'n'),
 ('END', 'END', 'END'),
 ('ROOT', 'ROOT', 'ROOT')]
149704
[('ROOT', 'ROOT', 'ROOT'),
 ('END', 'END', 'END'),
 ('ROOT', 'ROOT', 'ROOT'),
 ('W', 'poss.1sg', 'n'),
 ('END', 'END', 'END'),
 ('ROOT', 'ROOT', 'ROOT'),
 ('END', 'END', 'END'),
 ('ROOT', 'ROOT', 'ROOT'),
 ('END', 'END', 'END'),
 ('ROOT', 'ROOT', 'ROOT'),
 ('END', 'END', 'END'),
 ('ROOT', 'ROOT', 'ROOT'),
 ('DʒI', 'fut', 'v'),
 ('m', '1sg', 'v'),
 ('END', 'END', 'END')]


In [161]:
with open('Kamchatka_morphemes.pickle', 'rb') as f:
    kam_mor = pickle.load(f)
with open('Sebjan_morphemes.pickle', 'rb') as f:
    seb_mor = pickle.load(f)

In [8]:
from collections import Counter

seb_count = Counter(seb_mor)
kam_count = Counter(kam_mor)

In [9]:
# число слов в каждом
# в прошлый раз было 34933 на камчатке и 51148 в себъяне
print(seb_count[(0,)])
print(kam_count[(0,)])

49804
32778


ЛАДНО

In [113]:
kam_pd = pd.DataFrame({
                       'morpheme': [x[0] for x in kam_count if 'ROOT' not in x and 'END' not in x], 
                       'gloss': [x[1] for x in kam_count if len(x)==3],
                       'pos': [x[2] for x in kam_count if len(x)==3],
                       'k_count': [kam_count[x] for x in kam_count if len(x)==3]
    })

seb_pd = pd.DataFrame({
                       'morpheme': [x[0] for x in seb_count if len(x)==3], 
                       'gloss': [x[1] for x in seb_count if len(x)==3],
                       'pos': [x[2] for x in seb_count if len(x)==3],
                       's_count': [seb_count[x] for x in seb_count if len(x)==3]
    })

In [ ]:
seb_pd = seb_pd.sort_values(('s_count'), ascending=False)
kam_pd = kam_pd.sort_values(('k_count'), ascending=False)
seb_pd['gloss'] = seb_pd['gloss'].map(lambda x: x.strip('-='))
seb_pd['morpheme'] = seb_pd['morpheme'].map(lambda x: x.strip('-='))
kam_pd['gloss'] = kam_pd['gloss'].map(lambda x: x.strip('-='))
kam_pd['morpheme'] = kam_pd['morpheme'].map(lambda x: x.strip('-='))

In [117]:
kam_pd.head()

,gloss,k_count,morpheme,pos
120,nonfut,2956,R(E),v
283,pst,2712,RI,v
161,gnr,1891,WEːČ,v
296,3sg,1786,n(I),v
270,prog,1675,D,v


In [118]:
seb_pd.to_pickle('Sebjan_morphemes_only.pickle')
kam_pd.to_pickle('Kamchatka_morphemes_only.pickle')

In [10]:
seb_pd = pd.read_pickle('Sebjan_morphemes_only.pickle')
kam_pd = pd.read_pickle('Kamchatka_morphemes_only.pickle')

А теперь надо джойнить ))00

In [11]:
total = pd.merge(seb_pd, kam_pd, how='outer', on=['morpheme', 'gloss', 'pos'])

In [12]:
total = total.fillna(0)
total.to_pickle('morpheme_counts.pickle')

In [130]:
total = pd.read_pickle('morpheme_counts.pickle')

In [131]:
total.head()

,morpheme,gloss,pos,s_count,k_count
0,R(E),nonfut,v,2084.0,2956.0
1,E,ep,v,2080.0,1552.0
2,n(I),3sg,v,2069.0,1786.0
3,čE,pf.ptc,v,1895.0,211.0
4,Gr(E),hab,v,1747.0,41.0


In [132]:
total['s_expected'] = 49804*(total['s_count']+total['k_count'])/(82582)
total['k_expected'] = 32778*(total['s_count']+total['k_count'])/(82582)

total['k_semi_l'] = total['k_count']*np.log(total['k_count']/total['k_expected'])
total['s_semi_l'] = total['s_count']*np.log(total['s_count']/total['s_expected'])
total['k_semi_l'] = total['k_semi_l'].fillna(0)
total['s_semi_l'] = total['s_semi_l'].fillna(0)
total['log_l'] = 2*(total['s_semi_l'] + total['k_semi_l'])

# здесь есть вытаскивание минимального значения из двух столбцов
total['ell'] = total['log_l']/(82582*np.log(
        total['s_expected']*(total['s_expected']<total['k_expected'])+total['k_expected']*(total['k_expected']<total['s_expected'])
                                     )
                              )

del total['k_semi_l']
del total['s_semi_l']

total = total.sort_values('log_l', ascending=False)

In [5]:
del total['k_expected']
del total['s_expected']
total.head()

,morpheme,gloss,pos,s_count,k_count,log_l,ell
37,WEːČ,gnr,v,335.0,1891.0,1947.801162,0.003477
4,Gr(E),hab,v,1747.0,41.0,1452.048547,0.002678
7,RI,pst,v,1308.0,2712.0,1262.801694,0.002073
3,čE,pf.ptc,v,1895.0,211.0,935.525501,0.001684
17,D,prog,v,765.0,1675.0,834.398174,0.001470


In [134]:
total.to_pickle('total_morphemes.pickle')

In [3]:
total = pd.read_pickle('total_morphemes.pickle')

In [135]:
total.to_excel('morphemes_new.xlsx')

# разбираюсь с алломорфами

In [13]:
with open('morpheme_2.txt') as f:
    content = f.read().split('\n\n')
print(len(content))
pprint(content[39])

360
'\\lx =eː\n\\a =ej\n\\a =eːj\n\\a =aː\n\\a =oː\n\\ge =emph\n\\dt 29/Feb/2012'


In [32]:
morphs = {}
m_re = re.compile('\Wlx (\S+)\n')
g_re = re.compile('\Wge (\S+)\n')
a_re = re.compile('\Wa (\S+)\n')
for part in content:
    morph = re.search(m_re, part)
    if morph:
        morph = morph.group(1)
        gloss = re.search(g_re, part).group(1)
        morphs[(morph, gloss)] = re.findall(a_re, part)

In [41]:
for key in morphs:
    for allo in morphs[key]:
        res = total.loc[total['morpheme']==allo.strip('-=')].loc[total['gloss']==key[1]]
        if len(res)>0:
            print(key)
            print(res)

('-E', 'ep')
    gloss morpheme pos  s_count  k_count   s_freq  k_freq  s_expected  \
642    ep        i   v      1.0      0.0  0.00002     0.0    0.603085   

     k_expected  log_l  ell  
642    0.396915    NaN  NaN  
('-G', 'advr')
    gloss morpheme pos  s_count  k_count   s_freq  k_freq  s_expected  \
660  advr        k   v      1.0      0.0  0.00002     0.0    0.603085   

     k_expected  log_l  ell  
660    0.396915    NaN  NaN  
('-n(I)', '3sg')
    gloss morpheme pos  s_count  k_count   s_freq  k_freq  s_expected  \
573   3sg        n   v      1.0      0.0  0.00002     0.0    0.603085   

     k_expected  log_l  ell  
573    0.396915    NaN  NaN  
('-G(I)', 'vr')
    gloss morpheme pos  s_count  k_count   s_freq    k_freq  s_expected  \
681    vr        ŋ   v      1.0     12.0  0.00002  0.000366     7.84011   

     k_expected     log_l       ell  
681     5.15989  16.13729  0.000119  
('-ssO', 'dur')
    gloss morpheme pos  s_count  k_count   s_freq  k_freq  s_expected  \
40

Они ничего не делают

# разбираюсь с конвербами

In [48]:
cvbs = []
for corp in [seb_mor, kam_mor]:
    for morph in corp:
        if len(morph) > 1 and 'cvb' in morph[1]:
            cvbs.append(morph[1].strip('-='))
cvbs = set(cvbs)
pprint(cvbs)

{'ant.cvb',
 'caus.neg.cvb',
 'cond.cvb',
 'des.neg.cvb',
 'dur.cvb',
 'imm.cvb',
 'impf.cvb',
 'mult.cvb',
 'neg.cvb',
 'nmdl.cvb',
 'pf.cvb.Y',
 'purp.cvb',
 'sim.cvb',
 'trm.cvb',
 'vr.cvb.Y'}


In [49]:
cvb_pd = []
for cvb in cvbs:
    if len(cvb_pd)==0:
        cvb_pd = total.loc[total['gloss']==cvb]
        continue
    cvb_pd = pd.concat([cvb_pd, total.loc[total['gloss']==cvb]])
cvb_pd.head()

,gloss,morpheme,pos,s_count,k_count,s_expected,k_expected,log_l,ell
404,trm.cvb,klEːkE,v,4.0,0.0,2.412342,1.587658,4.045571,1.059761e-04
171,trm.cvb,dle,v,31.0,9.0,24.123417,15.876583,5.332687,2.335554e-05
230,trm.cvb,kEn,v,14.0,9.0,13.870965,9.129035,0.003029,1.658669e-08
269,pf.cvb.Y,An,v,10.0,0.0,6.030854,3.969146,10.113929,8.884064e-05
605,pf.cvb.Y,An,?,1.0,0.0,0.603085,0.396915,1.011393,-1.325398e-05


In [59]:
cvb_pd = cvb_pd.sort_values(['log_l'], ascending=False)

In [8]:
del cvb_pd['s_expected']
del cvb_pd['k_expected']

cvb_pd

,gloss,morpheme,pos,s_count,k_count,log_l,ell
12,sim.cvb,nIkEn,v,1083.0,59.0,739.842694,1.464707e-03
88,mult.cvb,ntEkEn,v,101.0,0.0,102.150680,3.351210e-04
20,ant.cvb,RIdʒI,v,721.0,237.0,95.300264,1.942508e-04
39,purp.cvb,DE,v,307.0,405.0,85.455394,1.833427e-04
138,impf.cvb,ŋsI,v,44.0,0.0,44.501286,1.884072e-04
26,cond.cvb,REk,v,488.0,197.0,35.655486,7.702568e-05
256,nmdl.cvb,ŋE,v,11.0,33.0,22.626085,9.579313e-05
29,cond.cvb,mI,v,442.0,223.0,10.754677,2.335654e-05
269,pf.cvb.Y,An,v,10.0,0.0,10.113929,8.884064e-05
171,trm.cvb,dle,v,31.0,9.0,5.332687,2.335554e-05


In [61]:
cvb_pd.to_pickle('converbs.pickle')

In [6]:
cvb_pd = pd.read_pickle('converbs.pickle')

In [62]:
cvb_pd.to_excel('converbs.xlsx')

# 1
это уже начато там
морфемы лежат в отдельном словаре, кажется; нужно приложить это знание на таблицу?
или создать уже наконец промежуточное представление всего

# 2
сочетания морфем

сначала надо вытащить представления (кажется это уже сделано)

пройтись по всему, растащить каждое слово на пары морфем, первая (ROOT, morph), последняя (morph, END)


Сочетания хи-квадратом vs. то же самое что с одиночными морфемами но для биграммов!!!11!

In [162]:
with open('Kamchatka_morphemes.pickle', 'rb') as f:
    kam_mor = pickle.load(f)
with open('Sebjan_morphemes.pickle', 'rb') as f:
    seb_mor = pickle.load(f)

In [163]:
seb_mor[:10]

[('ROOT', 'ROOT', 'ROOT'),
 ('END', 'END', 'END'),
 ('ROOT', 'ROOT', 'ROOT'),
 ('W', 'poss.1sg', 'n'),
 ('END', 'END', 'END'),
 ('ROOT', 'ROOT', 'ROOT'),
 ('END', 'END', 'END'),
 ('ROOT', 'ROOT', 'ROOT'),
 ('END', 'END', 'END'),
 ('ROOT', 'ROOT', 'ROOT')]

In [179]:
def remove_ep(corp):
    res = []
    for x in corp:
        if 'ep' not in x:
            res.append(x)
    return res

seb_mor = remove_ep(seb_mor)
kam_mor = remove_ep(kam_mor)

### лирическое отступление. это не используется

In [172]:
def list_afters(corp_m_list):
    res = {}
    for i in range(len(corp_m_list)-1): # делаю словарь прост
        if corp_m_list[i] in res:
            if 'ep' in corp_m_list[i+1]:
                res[corp_m_list[i]].append(corp_m_list[i+2])
            else:
                res[corp_m_list[i]].append(corp_m_list[i+1])
        else:
            if 'ep' in corp_m_list[i+1]:
                res[corp_m_list[i]] = [corp_m_list[i+2]]
            else:
                res[corp_m_list[i]] = [corp_m_list[i+1]]
    return res

In [173]:
from collections import Counter
# словарь морфема: список(каждый последующий), потом по списку считаю частотность
k_morph_dl = list_afters(kam_mor)
k_morph_pairs_dict = {x: Counter(k_morph_dl[x]) for x in k_morph_dl}

s_morph_dl = list_afters(seb_mor)
s_morph_pairs_dict = {x: Counter(s_morph_dl[x]) for x in s_morph_dl}

In [174]:
s_morph_pairs_dict[('WEːČ', 'gnr', 'v')]

Counter({('B', 'med', 'v'): 2,
         ('D', 'prog', 'v'): 47,
         ('DEŋ', 'pst.ptc', 'v'): 6,
         ('DU', 'dat', 'v'): 1,
         ('DʒI', 'fut', 'v'): 15,
         ('END', 'END', 'END'): 10,
         ('Gr(E)', 'hab', 'v'): 67,
         ('Gr(E)', 'hab.nonfut', 'v'): 5,
         ('L', 'inch', 'v'): 2,
         ('R', 'neg.cvb', 'v'): 11,
         ('R(E)', 'nonfut', 'v'): 19,
         ('REk', 'cond.cvb', 'v'): 3,
         ('RI', 'impf.ptc', 'v'): 33,
         ('RI', 'pst', 'v'): 35,
         ('RIdʒI', 'ant.cvb', 'v'): 5,
         ('W', 'advrs', 'v'): 1,
         ('dʒEːn', 'dur', 'v'): 1,
         ('j', 'conn', 'v'): 1,
         ('jEktEːk', 'nec.Y', 'v'): 2,
         ('m', 'des', 'v'): 2,
         ('mEČ', 'rec', 'v'): 2,
         ('mI', 'cond.cvb', 'v'): 15,
         ('n', 'nr', 'v'): 1,
         ('nE', 'intent', 'v'): 1,
         ('nIkEn', 'sim.cvb', 'v'): 26,
         ('ntEkEn', 'mult.cvb', 'v'): 2,
         ('sčI', 'conat', 'v'): 1,
         ('čE', 'pf.ptc', 'v'): 19})

### теперь снова осмысленное

In [180]:
# bigramm-style!
k_pairs = Counter([(kam_mor[i], kam_mor[i+1]) for i in range(len(kam_mor)-1)])
s_pairs = Counter([(seb_mor[i], seb_mor[i+1]) for i in range(len(seb_mor)-1)])

kpd_pairs = pd.DataFrame({
                       '1morpheme': [x[0][0] for x in k_pairs], 
                       '1gloss': [x[0][1] if len(x[0])==3 else x[0][0] for x in k_pairs],
                       '1pos': [x[0][2] if len(x[0])==3 else x[0][0] for x in k_pairs],
                       '2morpheme': [x[1][0] for x in k_pairs], 
                       '2gloss': [x[1][1] if len(x[1])==3 else x[1][0] for x in k_pairs],
                       '2pos': [x[1][2] if len(x[1])==3 else x[1][0] for x in k_pairs],
                       'k_count': [k_pairs[x] for x in k_pairs]
    })

spd_pairs = pd.DataFrame({
                       '1morpheme': [x[0][0] for x in s_pairs], 
                       '1gloss': [x[0][1] if len(x[0])==3 else x[0][0] for x in s_pairs],
                       '1pos': [x[0][2] if len(x[0])==3 else x[0][0] for x in s_pairs],
                       '2morpheme': [x[1][0] for x in s_pairs], 
                       '2gloss': [x[1][1] if len(x[1])==3 else x[1][0] for x in s_pairs],
                       '2pos': [x[1][2] if len(x[1])==3 else x[1][0] for x in s_pairs],
                       's_count': [s_pairs[x] for x in s_pairs]
    })

kpd_pairs.head()

,1gloss,1morpheme,1pos,2gloss,2morpheme,2pos,k_count
0,advr,pčI,pron,END,END,END,4
1,acclr,mElčI,v,res,Č,v,1
2,ROOT,ROOT,ROOT,emph,ńUn,adv,4
3,prfl.pl,WUr,n,emph,gEli,n,1
4,pl,L,pron,loc,(dU)LE,pron,2


In [181]:
total_pairs = pd.merge(spd_pairs, kpd_pairs, how='outer', on=['1morpheme', '1gloss', '1pos', '2morpheme', '2gloss', '2pos'])

# for column in ['1morpheme', '1gloss', '1pos', '2morpheme', '2gloss', '2pos']:
#     total_pairs[column] = total_pairs[column].map(lambda x: x.strip('-=') if x not in [0, 1] else x)

total_pairs = total_pairs.sort_values(['1gloss', '1morpheme', '1pos'])

total_pairs = total_pairs.loc[total_pairs['s_count'] != 49803]
total_pairs = total_pairs.loc[total_pairs['1gloss'] != 'ROOT']
total_pairs = total_pairs.loc[total_pairs['1gloss'] != '']
total_pairs = total_pairs.loc[total_pairs['1gloss'] != '*']
total_pairs = total_pairs.loc[total_pairs['1gloss'] != '***']
total_pairs = total_pairs.loc[total_pairs['1gloss'] != '******']

total_pairs.head()

,1gloss,1morpheme,1pos,2gloss,2morpheme,2pos,s_count,k_count
618,1pl.Y,BIt,ptl,END,END,END,1.0,NaN
2450,1pl.Y,BIt,v,END,END,END,5.0,NaN
3120,1pl.ex,(R)U,?,END,END,END,NaN,4.0
4548,1pl.ex,(R)U,n,END,END,END,NaN,1.0
3366,1pl.ex,(R)U,v,ptl,tIt,v,NaN,1.0


In [182]:
total_pairs.to_pickle('pairs.pickle')

## примерно здесь заканчивается разумное. goto pairs.ipynb

49804
32778

In [83]:
total_pairs = pd.read_pickle('pairs.pickle')

In [69]:
total_pairs['s_count'] = total_pairs['s_count'].fillna(0)
total_pairs['k_count'] = total_pairs['k_count'].fillna(0)

In [75]:
total_pairs['s_expected'] = 49804*(total_pairs['s_count']+total_pairs['k_count'])/(82582)
total_pairs['k_expected'] = 32778*(total_pairs['s_count']+total_pairs['k_count'])/(82582)
total_pairs['k_semi_l'] = total_pairs['k_count']*np.log(total_pairs['k_count']/total_pairs['k_expected'])
total_pairs['s_semi_l'] = total_pairs['s_count']*np.log(total_pairs['s_count']/total_pairs['s_expected'])
total_pairs['k_semi_l'] = total_pairs['k_semi_l'].fillna(0)
total_pairs['s_semi_l'] = total_pairs['s_semi_l'].fillna(0)
total_pairs['log_l'] = 2*(total_pairs['s_semi_l']+total_pairs['k_semi_l'])
total_pairs['ell'] = total_pairs['log_l']/(82582*np.log(
        total_pairs['s_expected']*(total_pairs['s_expected']<total_pairs['k_expected'])+total_pairs['k_expected']*(total_pairs['k_expected']<total_pairs['s_expected'])
                                     )
                              )
del total_pairs['k_semi_l']
del total_pairs['s_semi_l']
total_pairs = total_pairs.sort_values('log_l', ascending=False)
total_pairs.head()

,1gloss,1morpheme,1pos,2gloss,2morpheme,2pos,s_count,k_count,s_expected,k_expected,log_l,ell
1558,gnr,WEːČ,v,nonfut,R(E),v,19.0,1506.0,919.705263,605.294737,2598.003320,0.004911
4680,prog,D,v,gnr,WEːČ,v,0.0,458.0,276.213122,181.786878,846.415309,0.001970
2445,sim.cvb,nIkEn,v,1,1,1,1039.0,34.0,647.110654,425.889346,812.034418,0.001624
999,pf.ptc,čE,v,1,1,1,1083.0,73.0,697.166743,458.833257,685.667361,0.001355
4769,poss.1pl.ex,WUn,v,1,1,1,0.0,327.0,197.208932,129.791068,604.318354,0.001504


In [118]:
total_pairs = pd.read_pickle('pairs_by_ll.pickle')
total_pairs.head(20)

,1gloss,1morpheme,1pos,2gloss,2morpheme,2pos,s_count,k_count,s_expected,k_expected,log_l,ell
1558,gnr,WEːČ,v,nonfut,R(E),v,19.0,1506.0,919.705263,605.294737,2598.003320,0.004911
4680,prog,D,v,gnr,WEːČ,v,0.0,458.0,276.213122,181.786878,846.415309,0.001970
2445,sim.cvb,nIkEn,v,1,1,1,1039.0,34.0,647.110654,425.889346,812.034418,0.001624
999,pf.ptc,čE,v,1,1,1,1083.0,73.0,697.166743,458.833257,685.667361,0.001355
4769,poss.1pl.ex,WUn,v,1,1,1,0.0,327.0,197.208932,129.791068,604.318354,0.001504
4394,pst,RI,v,poss.1pl.ex,WUn,v,0.0,318.0,191.781163,126.218837,587.685739,0.001471
3384,1pl.ex,(R)U,v,1,1,1,0.0,295.0,177.910198,117.089802,545.180166,0.001386
707,pst,RI,v,poss.3sg,n(I),v,641.0,1145.0,1077.110557,708.889443,432.596081,0.000798
1301,hab,Gr(E),v,3pl,r,v,417.0,0.0,251.486619,165.513381,421.750828,0.001000
698,prog,D,v,pst,RI,v,27.0,301.0,197.812017,130.187983,397.014827,0.000987


In [77]:
total_pairs.to_pickle('pairs_by_ll.pickle')
total_pairs.to_excel('pairs_by_ll.xlsx')

In [86]:
pairs = pd.read_pickle('pairs_by_ll.pickle')   
del pairs['s_expected']
del pairs['k_expected']
del pairs['log_l']
del pairs['ell']

,1gloss,1morpheme,1pos,2gloss,2morpheme,2pos,s_count,k_count
1558,gnr,WEːČ,v,nonfut,R(E),v,19.0,1506.0
4680,prog,D,v,gnr,WEːČ,v,0.0,458.0
2445,sim.cvb,nIkEn,v,1,1,1,1039.0,34.0
999,pf.ptc,čE,v,1,1,1,1083.0,73.0
4769,poss.1pl.ex,WUn,v,1,1,1,0.0,327.0


In [88]:
len(s_morph_pairs_dict)

726

In [101]:
# вернёмся к вложенной структуре
# k_morph_pairs_dict
# s_morph_pairs_dict
from scipy import stats


def cramers_corrected_stat(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher, 
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min( (kcorr-1), (rcorr-1)))


def get_chi(obs_s, obs_k):
    obs = np.array([obs_s, obs_k])
    chi2, p, dof, expected = stats.chi2_contingency(obs)
    iff = expected.reshape(1, expected.shape[1]*2)[0]
    if all(iff>5):
        df = pd.DataFrame({'s': obs_s, 'k': obs_k})
        confusion_matrix = pd.crosstab(df['s'], df['k'])
        print(confusion_matrix)
        print(df)
        c_v = cramers_corrected_stat(confusion_matrix)
        info = (chi2, dof, p, c_v)
        return info
    return ()


tuples = []
observe_s = []
observe_k = []
staty = []
for key in k_morph_pairs_dict:
    if key in s_morph_pairs_dict:
        lines = list(set(k_morph_pairs_dict[key].keys())|set(s_morph_pairs_dict[key].keys()))
        n = len(lines)
        obs_s = [s_morph_pairs_dict[key][x] if x in s_morph_pairs_dict[key] else 0 for x in lines]
        obs_k = [k_morph_pairs_dict[key][x] if x in k_morph_pairs_dict[key] else 0 for x in lines]
        lines = list(map(lambda x: ' '.join([str(x[0]), str(x[0]), str(x[0])]) if len(x)==1 else ' '.join(x), lines))
        key = list(map(lambda x: ' '.join([str(x[0]), str(x[0]), str(x[0])]) if len(x)==1 else ' '.join(x), [key]))
        index = list(zip(key*n, lines))
        if n>1:
            info = get_chi(obs_s, obs_k)
        else:
            info = (0, 0, 1, -1)
        if len(info)>0:
            tuples += index
            observe_s += obs_s
            observe_k += obs_k
            staty += [info]*n

a_index = pd.MultiIndex.from_tuples(tuples, names=['morph1', 'morph2'])
tab = pd.DataFrame({'count_s': observe_s, 'count_k': observe_k, 'chi2': list(map(lambda x: x[0], staty)), 
                   'dof': list(map(lambda x: x[1], staty)), 'p': list(map(lambda x: x[2], staty)), 
                   'c_v': list(map(lambda x: x[3], staty))}, index=a_index)
tab.sort_values(['p']).head()

,,c_v,chi2,count_k,count_s,dof,p
morph1,morph2,,,,,,
-LE -prv ?,1 1 1,-1,0,5,1,0,1
=eː =emph quant,1 1 1,-1,0,16,3,0,1
-tEn -poss.3pl num,1 1 1,-1,0,1,4,0,1
=GU =q n,1 1 1,-1,0,7,44,0,1
-n(I) -poss.3sg postpos,1 1 1,-1,0,1,7,0,1


In [107]:
def get_fi(obs_s, obs_k):
    obs = np.array([obs_s, obs_k])
#     chi2, p, dof, expected = stats.chi2_contingency(obs)
    oddsratio, p = stats.fisher_exact(obs)
#     iff = expected.reshape(1, expected.shape[1]*2)[0]
#     print(expected)
#     if all(iff>5):
    df = pd.DataFrame({'s': obs_s, 'k': obs_k})
    confusion_matrix = pd.crosstab(df['s'], df['k'])
    print(confusion_matrix)
    print(df)
    c_v = cramers_corrected_stat(confusion_matrix)
    info = (oddsratio, p, c_v)
    return info
#     return ()


for key in k_morph_pairs_dict:
    if key in s_morph_pairs_dict:
        lines = list(set(k_morph_pairs_dict[key].keys())|set(s_morph_pairs_dict[key].keys()))
        n = len(lines)
        obs_s = [s_morph_pairs_dict[key][x] if x in s_morph_pairs_dict[key] else 0 for x in lines]
        obs_k = [k_morph_pairs_dict[key][x] if x in k_morph_pairs_dict[key] else 0 for x in lines]
        lines = list(map(lambda x: ' '.join([str(x[0]), str(x[0]), str(x[0])]) if len(x)==1 else ' '.join(x), lines))
        key = list(map(lambda x: ' '.join([str(x[0]), str(x[0]), str(x[0])]) if len(x)==1 else ' '.join(x), [key]))
        index = list(zip(key*n, lines))
        lines = list(map(lambda x: ' '.join([str(x[0]), str(x[0]), str(x[0])]) if len(x)==1 else ' '.join(x), lines))
        key = list(map(lambda x: ' '.join([str(x[0]), str(x[0]), str(x[0])]) if len(x)==1 else ' '.join(x), [key]))
        index = list(zip(key*n, lines))
        if n>1:
            info = get_fi(obs_s, obs_k)
        else:
            info = (0, 1, -1)
        if len(info)>0:
            tuples += index

ValueError: The input `table` must be of shape (2, 2).

In [125]:
with open('Sebjan.pickle', 'rb') as f:
    seb = pickle.load(f)
with open('Kamchatka.pickle', 'rb') as f:
    kam = pickle.load(f)
for corp in [seb, kam]:
    i = 0
    for doc in corp:
        for sent in corp[doc]['text']:
            if 'ps' in sent:
                if ['conj'] in sent['ps']:
                    i += 1
    print(i)

235
404


In [127]:
with open('Kamchatka_pairs_dict.pickle', 'wb') as f:
    pickle.dump(k_morph_pairs_dict, f)
with open('Sebjan_pairs_dict.pickle', 'wb') as f:
    pickle.dump(s_morph_pairs_dict, f)